In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [78]:
url = requests.get('https://www.carpages.ca/used-cars/search/?num_results=50&fueltype_id%5B0%5D=3&fueltype_id%5B1%5D=7&p=10')

In [79]:
soup = BeautifulSoup(url.text, 'lxml')

In [80]:
posting = soup.find_all('div', class_ = 'media soft push-none rule')

In [83]:
count = 10
key = True
while key:
    data_dict = {'link' : [], 'name' : [], 'price' : []}
    for post in posting:
        data_dict['link'].append('https://www.carpages.ca'+post.find('a', class_ = 'media__img media__img--thumb').get('href'))
        data_dict['name'].append(post.find('h4', class_ = 'hN').text.strip())
        data_dict['price'].append(post.find('strong', class_ = 'delta').text.strip())
    df1 = pd.DataFrame(data_dict)
    df = pd.concat([df, df1])
    count = count + 1
    if count == 15:
        key = False
    else:
        next_page_link = 'https://www.carpages.ca'+soup.find('a', {'title':'Next Page'}).get('href')
        next_page_url = requests.get(next_page_link)
        soup = BeautifulSoup(next_page_url.text, 'lxml')
        posting = soup.find_all('div', class_ = 'media soft push-none rule')

In [85]:
df.to_excel('carpage_scraping.xlsx')